<h1><center> Laboratorio di WebScraping </h1>
<h1><center> Anno Accademico 2024-2025 </h1>
<h1><center>  Docente: Laura Ricci </h1>
<h1><center>  Lezione 13 </h1>
<h1><center>  Web Scraping:  Forms e Pagine Dinamiche </h1>
<h1><center> 26 Marzo 2025 </h1>

## Scraping Hacker News 

* **https://news.ycombinator.com/news**
* un sito **aggregatore** di news reputate interessante dalla comunità degli hacker
    * computer scientists, data scientists, engineers
* la struttura della pagina è molto semplice
* utilizzeremo delle **Regex** per individuare il contenuto

<center>
<img src="Figures/HackerNewsNew.jpg" style="width:800px;height:600px;"/>
</center>

## Scraping Hacker News: analisi HTML

<center>
<img src="Figures/HackerNewsInspectN.jpg" style="width:800px;height:600px;"/>
</center>

* una tabella in cui ogni riga (**tag \<tr\>**), ha **classe athing submission**
* un elemento della tabella (**tag \<td\>**), ha **classe titleline**, contiene descrizione della notizia e link alla notizia

## Scraping Hacker News con BeautifulSoup

* reperimento titolo e link
* reperire score e numero di commenti, 
* for separato per le due coppie di informazioni precedenti, solo per rendere più chiara la presentazione
* inseriamo informazioni reperite in un **Python dictionary**

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
articles = []
url = 'https://news.ycombinator.com/news'
r = requests.get(url)
html_soup = BeautifulSoup(r.text, 'html.parser')

## Scraping Hacker News con BeautifulSoup

* ricerco tutte le righe della tabella
*  all'interno di ogni riga, cerco titolo e riferimenyo
* enormi pontenzialità se poi si eseguisse del text mining sul contenuto dei post

In [ ]:
for item in html_soup.find_all('tr', class_='athing'):
    item_titlel = item.find('span', class_='titleline')
    item_title = item_titlel.find('a')
    item_title = item_title.get_text(strip=True) if item_title else 'No Link'
    item_link = item_titlel.a.get("href")
    articles.append({
        'link' : item_link,
        'title' : item_title,
         })


## Scraping Hacker News con BeautifulSoup

<center>
<img src="Figures/HackeNewsInspectN1.jpg" style="width:700px;height:600px;"/>
</center>

## Scraping Hacker News con BeautifulSoup

In [ ]:
scores=[]
for item in html_soup.find_all('tr', class_='athing'):
    next_row = item.find_next_sibling('tr')
    item_score = next_row.find('span', class_='score')
    item_score = item_score.get_text(strip=True) if item_score else '0 points'
    item_comments = next_row.find('a', string=re.compile('\\d+(&nbsp;|\\s)comment(s?)'))
    item_comments = item_comments.get_text(strip=True).replace('\xa0', ' ') \
                    if item_comments else '0 comments'
    scores.append({
        'score' : item_score,
        'comments' : item_comments
         })
result = zip(articles[:3], scores[:3])
for res in result:
    print(res)

## Utilizzo di Regex:  \\d+(\&nbsp;|\\s)comment(s?)

* **\\d+**
   * \\d  corrisponde a una qualsiasi cifra (0-9)
   * è necessario usare la doppia backslash (\\d) perchè il singolo carattere è usato come carattere di escape nelle stringhe
   * +: **una o più occorrenze** del carattere precedente
   * \\d+ corrisponde a uno o più cifre (e.g., "1", "23", "456").

* **(\&nbsp;|\\s)**
   * | è l'operatore **OR**, quindi è indica che è corretta una corrispondenza con l'operatore di destra o con quello di sinistra
   * &nbsp: HTML non-breaking space 
   * \\s corrisponde a un carattere, un tab, una newline


## Utilizzo di Regex:  \\d+(\&nbsp;|\\s)comment(s?)

* **"comment"** si cerca una corrispondenza esatta con la parole **comment**

*  **(s?)**
   * s? significa che il carattere  "s" è opzionale (? corrisponde a 0 o più di una occorrenza)
   * quindi va bene al termine della stringa sia la parole "comment" che la parola "comments".

 ## Utilizzo di Regex:  \\d+(&nbsp;|\\s)comment(s?)
 
* possibili corrispondenza
    
    * 4 comments
    * 16&nbsp;comments
    * 120 comment
    * 21 comments
    * 1&nbsp;comment

## Non-breaking space (&nbsp in HTML) 

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Non-Breaking Space Example</title>
</head>
<body>
    <p>Without non-breaking space: 100 km</p>
    <p>With non-breaking space: 100&nbsp;km</p>
</body>
</html>

* nella prima linea, "100 km" è possibile che "100" e "km." siano separati da un " a capo"
* nella seconda linea,  "100" e "km" non sarabnno mai separati

 ## Utilizzo della operazione Replace
    
* l'esperessione **('\xa0', ' ')** è utilizzata per rimpiazzare le occorrenze di \xa0 ( non-breaking space) 
  con la stringa ' ' 
*  \xa0 rappresenta non-breaking space (NBSP) ed è comunemente utilizzato nelle pagine web e in **HTML**
* il metodo **replace** sostituisce tutte le occorrenze del primo argomneto con il secondo argomento

In [ ]:
text = "Hello\xa0World"
print(text)
clean_text = text.replace('\xa0', ' ')
print(clean_text)  

## Strumenti avanzati per il web scraping

* le tecniche viste nelle lezioni precedenti possono essere utilizzate sotto certe condizioni
    * la pagina **HTML** che cerchiamo è accessibile senza autenticazione
    * il contenuto della pagina è **statico**
    * il server non utilizza misure anti-scraping.
* cosa manca per implementare un vero web-scraper?
    * gestione di web forms per siti che richiedono autenticazione
    * gestione di cookies
    * capire le misure opportune affinchè lo scraper **looks like a human** 
    * gestire il contenuto dinamico generato da **JavaScript**
* alcune di queste tecniche possono essere implementate direttamente in **BeautifulSoup**
* la gestione di contenuti gestiti da **Java Script** richiede strumenti più avanzati, come **Selenium**
    

## Interagire con un web server: query strings

* **HTTP** consente non solo di scaricare contenuti dal server, ma anche di interagire con il server inviandogli dei dati
* il modo più semplice per inviare i dati a un server è includere i dati nella **URL**
* **query strings**: parte opzionale della **URL** inserita dopo il **?** 
    <code>
    http://www.example.com/product_page.html?product_id=304
    </code>
     <code>
    https://www.google.com/search?dcr=0&source=hp&q=test&oq=test
    </code>
* quando il server riceve una richiesta **HTTP** con una query string, è in grado di "interpretare i parametri" ed effettuare le operazioni corrispondenti
* anche se la dimensione di una **URL** è illimitata, inviare parametri tramite una **URL** diventa difficile, quando le query diventano più complesse
    * ad esempio: comprare tickets per un concerto,  invinado nome, e-mail, scelta del ticket, etc.

## HTML Form

* una form (modulo) è una sezione di documento HTML che contiene **elementi di controllo** che l’utente può utilizzare per inserire 
dati o in generale per interagire col server
* i dati inseriti possono essere poi inoltrati al server dove **un agente** può elaborarli
* gli elementi di controllo sono caratterizzati da un valore iniziale e da un valore corrente
* gli elementi di controllo possono essere:
     * caselle di inserimento di testo
     * bottoni di azione
     * checkbox (caselle di spunta)
     * radio Button (bottoni mutuamente esclusivi)Liste di selezione (lista di opzioni)
     * oggetti nascosti (elementi valorizzati ma invisibili)

## Il tag \<form\>

* una **HTML Form (Web Form)** è incorporata nella pagina **HTML** mediante i tag  **&lt;form>.. &lt;/form>**

* all'interno della form si possono trovare diversi attributi che definiscono la **form**
    * **action = url**
        * URL dell’agente che riceverà i dati del form, quindi dove devono essere inviati i dati
        * il suo valore deve essere un URL valido, relativo o assoluto. 
        * se questo attributo non viene fornito, i dati verranno inviati all'URL della pagina che contiene il form, ovvero        la pagina corrente.
    * **name = text**: specifica il nome della form
    * **method = {get|post}**: specifica il modo in cui i dati vengono inviati
    * **enctype = content-type**: se il metodo è POST specifica il content type usato per la codifica (encoding) dei dati contenuti       nella form;
* ad esempio:
<code>
    &lt;form action="http://site.com/bin/adduser“ method="post">
    ...form contents...
    &lt;/form> 
</code>

## Elementi di Input

* la maggior parte dei controlli che consentono l'interazione con la form vengono definiti  mediante il tag **&lt;input>**
* l’attributo **type** di questo tag stabilisce il tipo di controllo
    * text: casella di testo monoriga
    * password: come text ma il testo non è leggibile (****)
    * file: controllo che consente di caricare un file
    * checkbox: casella di spunta
    * radio: radio button
    * submit: bottone per trasmettere il contenuto del form
    * image: bottone di submit sotto forma di immagine
    * reset: bottone che riporta tutti i campi al valore iniziale
    * button: bottone di azione
    * hidden: campo nascosto

## Input text

* è un campo per l’inserimento di testo su una sola riga
* attributi:
  * name = text. nome del controllo
  * value = text, eventuale valore iniziale
  * size = n. lunghezza del campo (numero di caratteri), maxlength = n
  * massima lunghezza del testo (numero di caratteri)
<code>  
&lt;form action="http://site.com/bin/adduser" method="post">
   &lt;p>
     Nome: &lt;input type="text" name="firstname">
    &lt;p>
&lt;form>
</code>


## Gestire una form di login: https://practice.expandtesting.com/login

<img src="Figures/FormLogin.jpg" style="width:1000px;height:300px;"/>

## Gestire una form di login: https://practice.expandtesting.com/login

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://practice.expandtesting.com/login'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
form = soup.find('form')
form_data = {}
for input_tag in form.find_all('input'):
    name = input_tag.get('name')
    value = input_tag.get('value', '')
    form_data[name] = value

print(form_data) # {'username': '', 'password': ''}

## Gestire una form di login: https://practice.expandtesting.com/login

* utilizzare la **post** per inviare i dati al server
* individuare dal campo action dove devono essere inviati i dati
* requests.compat.urljoin(url, action_url) combina una **URL base**   con una **URL relativa** (action_url) per creare una **URL assoluta**

In [ ]:
form_data['username'] = 'practice'
form_data['password'] = 'SuperSecretPassword!'

action_url = form.get('action')
# Construct the full URL if the action is a relative URL
if not action_url.startswith('http'):
    action_url = requests.compat.urljoin(url, action_url)
response = requests.post(action_url, data=form_data)
print(response.status_code)
print(response.text)

## Gestire una form: http://testphp.vulnweb.com/login.php

<img src="Figures/LoginForm.jpg" style="width:1200px;height:600px;"/>

## Gestire le form

<img src="Figures/HTMLForm.jpg" style="width:800px;height:800px;"/>

## Web Form: attributi

* **method**
    * specifica il metodo **HTTP**  usato per la sottomissione dei dati immessi dall'utente
        * default method **GET** con parametri embedded nella **URL**
    * se settato a **POST** il browser esegue una **HTTP POST** per inviare i dati, invece di inserire i dati nell **URL** (metodo **GET**)
  <code>
    &lt;form method="post"> 
    &lt;/form>
    </code>  
    * viene inviata una richiesta **HTTP POST**
        * il valore dei parametri viene incluso nel body della richiesta, invece di includere il valore dei parametri nella **URL** di un metodo **GET**
* **action**
    * indica la **URL** dove i dati verranno mandati e gestiti una volta che il contenuto del form verrà inviato.
     

## Pagina acceduta dopo il login

<img src="Figures/AfterLogin.jpg" style="width:800px;height:600px;"/>

## Pagina acceduto dopo il login

In [ ]:
from bs4 import BeautifulSoup as bs 
import requests 
URL = "http://testphp.vulnweb.com/userinfo.php" 
payload = { 
 "uname": "test", 
 "pass": "test" 
} 
response = requests.get(URL)
response = requests.post(URL, data=payload) 
print(response.status_code) # If the request went Ok we usually get a 200 status. 
soup = BeautifulSoup(response.content, "html.parser") 
content = soup.find(attrs={"id": "pageName"}).text 
print(content)

## Usare gli  Strumenti per sviluppatori del browser

* usare il tab **Strumenti per sviluppatori** di Chrome per monitorare cosa accade quando si sottomette la **form**
* usare **test** come username e password
* premere il bottone **login** e controllare la richiesta che il browser invia al server
* la sottomissione della **form** genera una richiesta **POST** alla pagina individuata
* il server risponde con un **cookie** e permette l'accesso alla pagina protetta

## Usare gli  Strumenti per sviluppatori del browser: intestazione

<img src="Figures/Intestazioni.jpg" style="width:800px;height:600px;"/>

## Usare gli  Strumenti per sviluppatori del browser

<img src="Figures/Payload.jpg" style="width:1400px;height:600px;"/>

* metodo **POST** della libreria *Request**
* dizionario che rappresenta coppie nome, valore


## La tecnologia AJAX

* la  maggior parte dei siti usa tecnologie di tipo **JavaScript** o **AJAX**
* **AJAX: Asynchronous JavaScript and XML**
  * un insieme di tecniche per lo sviluppo web, non un vero linguaggio di programmazione
  * da la possibilità di effettuare richieste **asincrone** al server, invece   di richieste **HTTP** classiche, utilizzando chiamate del tipo **XMLHttpRequest** 
  * l'informazione reperita può essere utilizzata per l'aggiornamento **dinamico** di una pagina
* possibilità di **aggiungere** o **aggiornare** il contenuto di una pagina web, senza dover effettuare un **refresh** della pagina
* per migliorare la **user experience**
* uso tipico: siti che incrementano dinamicamente il contenuto mostrato quando si effettua lo **lo scrolling** della pagina
* qualsiasi social network **Facebook**, **Twitter** utilizzano la tecnica di **infinite scroll pagination**
    * per implementare il **social media feed**
    * quando si effettua uno **scroll down** si carica più contenuto (più post) nella pagina

## La tecnologia AJAX

* pagina di esempio https://scrapingclub.com/exercise/list_infinite_scroll/
* scrollando la pagina si può notare come i capi di abbigliamento vengono caricati dinamicamente

<center>
<img src="Figures/InfiniteScroll.jpg" style="width:600px;height:600px;"/>
</center>

## Scraping di pagine dinamiche

In [1]:
import requests
from bs4 import BeautifulSoup
url = 'https://scrapingclub.com/exercise/list_infinite_scroll/'
response = requests.get(url)
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="h-full">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="Learn to scrape infinite scrolling pages" name="description"/>
  <title>
   Scraping Infinite Scrolling Pages (Ajax) | ScrapingClub
  </title>
  <link href="/static/img/icon.611132651e39.png" rel="icon" type="image/png"/>
  <script>
   window.django_app_cfg = {
        dsn: "https://547e0f728838410daf2f1e1a20153fcc@o207778.ingest.sentry.io/4505046779232256",
        release: "2c93d99",
        userPk: "",
      };
  </script>
  <link href="/static/css/app.4a8ac9e71a688f8d087e.76f3a3e8d3f5.css" rel="stylesheet" type="text/css"/>
  <!-- Google tag (gtag.js) -->
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-BD9ZHFE1XX">
  </script>
  <script>
   window.dataLayer = window.dataLayer || [];

      function gtag() {
        dataLayer.push(arguments);
      }

      gtag('js', new Date());

      gtag('config', 

## Scraping di pagine dinamiche

<center>
<img src="Figures/InfiniteScrollJS.jpg" style="width:600px;height:600px;"/>
</center>

* il contenuto dinamico non è presente nella pagina **HTML**
* individuiamo nella pagina la parte relativa allo scroll: si tratta di codice **JavaScript**, che interpreta lo scrolling a e  manda chiamate **AJAX** al server
* come lo acquisisco
    * inutile cercare un link nella pagina per passare alla pagina successiva

## Scraping di pagine dinamiche

* è possibile effettuare ancora scraping di questa pagina con **BeutifulSoup**?
* in questo particolare caso, è ancora possibile
    * individuare le chiamate **AJAX** effettuate dal browser per reperire dinamicamente le pagine
    * replicare tali chiamate in Python usando la ribreria **Requests**
    * dopo aver caricato tutte le pagine (che si otterebbero con scroll della pagina) parsare il contenuto

## Pagine dinamiche: https://scrapingclub.com/exercise/list_infinite_scroll/

<center>
<img src="Figures/InspectInfiniteScroll.jpg" style="width:800px;height:400px;"/>
</center>

## Il tab network

* durante le operazioni di scraping particolare importanza ha il **tab Network** 
* selezionato il tab **Network** la finestra che si apre mostra tutte le richieste effettuate dal browser (e quindi dalla pagina web) al server durante il caricamento.
    * in **Chrome** selazionare **More Tools**, **Developer Tools**, quindi il tab **Network**

<center>
<img src="Figures/TabNetwork.jpg" style="width:1000px;height:300px;"/>
</center>

## Il tab network

* il tag **XHR** filtra e mostra le  **XMLHttpRequests**
    * utilizzato per monitorare le chiamate **AJAX** effettuate al server background, mentre il client sta lavorando in foreground, 
    e aggiornare la pagina con i dati ricevuti, senza ricaricare l'intera pagina
    * **XML** nel nome è un pò fuorvinate, può essere utilizzato per scambiare dati in altri formati, ad esempio **JSON**

<center>
<img src="Figures/TabNetwork.jpg" style="width:1000px;height:300px;"/>
</center>

## Identificare richieste AJAX effettuate dal browser

<center>
<img src="Figures/XHRRequests.jpg" style="width:1300px;height:300px;"/>
</center>

* nel caso del nostro sito refresh della pagina in seguito a scroll
    * viene generata una nuova richiesta per ogni nuovo contenuto caricato quando si effettua lo scroll della pagina
    * la **URL** delle richieste può essere selezionata nelle informazioni visualizzate selezionando questo tag
    * individuare la **URL** di ogni richiesta e poi copiarla per effettuare richieste tramite la libreria **requests**


##  Scraping di pagine dinamiche: identificare richieste AJAX

<center>
<img src="Figures/XHRRequests.jpg" style="width:1300px;height:300px;"/>
</center>

* dalla analisi delle richieste AJAX si nota che 
    * ci sono 6 pagine in totale, corrispondenti ai diversi scroll di pagina
* ogni richiesta **XMLHttpRequests** ha il formato  
    * **[base_url] + [page_number]**, dove **[base_url]** è https://scrapingclub.com/exercise/list_infinite_scroll/

## Pagine dinamiche: https://scrapingclub.com/exercise/list_infinite_scroll/

* individuare i nomi dei prodotti e i prezzi di ogni articolo

<center>
<img src="Figures/SelectDress.jpg" style="width:1200px;height:400px;"/>
</center>

## Scraping di pagine dinamiche

In [1]:
import requests
from bs4 import BeautifulSoup
base_url = 'https://scrapingclub.com/exercise/list_infinite_scroll/'
page_number = 1  # Start with the base URL
total_pages = 6 
while True:
    url = f'{base_url}?page={page_number}'
    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    products = soup.select('div.p-4 h4 > a')  
    prices =  prices = soup.select('div.p-4 h5') 
    for product, price in zip(products, prices):
        product_name = product.get_text(strip=True)
        product_price = price.get_text(strip=True)
        print(f'Product: {product_name} | Price: {product_price}')
    print('-' * 50)
    page_number += 1 
    if page_number > total_pages:
        break 

Product: Short Dress | Price: $24.99
Product: Patterned Slacks | Price: $29.99
Product: Short Chiffon Dress | Price: $49.99
Product: Off-the-shoulder Dress | Price: $59.99
Product: V-neck Top | Price: $24.99
Product: Short Chiffon Dress | Price: $49.99
Product: V-neck Top | Price: $24.99
Product: V-neck Top | Price: $24.99
Product: Short Lace Dress | Price: $59.99
Product: Fitted Dress | Price: $34.99
--------------------------------------------------
Product: V-neck Jumpsuit | Price: $69.99
Product: Chiffon Dress | Price: $54.99
Product: Skinny High Waist Jeans | Price: $39.99
Product: Super Skinny High Jeans | Price: $19.99
Product: Oversized Denim Jacket | Price: $19.99
Product: Short Sweatshirt | Price: $24.99
Product: Long-sleeved Jersey Top | Price: $12.99
Product: Skinny High Waist Jeans | Price: $39.99
Product: Short Sweatshirt | Price: $24.99
Product: Long-sleeved Jersey Top | Price: $12.99
--------------------------------------------------
Product: Long-sleeved Jersey Top | P

## BeautifulSoup: utilizzo dei CSS Selectors

* nell'esercizio precedente sono stati utilizzati dei **CSS Selectors**
* il metodo **select** consente di selezionare contenuti in base a **CSS selectors**
* selezionano un elemento **HTML** in base a
   * **nome del tag**
   * **nome della classe**
   * **identificatore**
   * **relazioni gerarchiche**

## CSS: relazioni gerarchiche

In [2]:
from bs4 import BeautifulSoup
example_html = """
    <!DOCTYPE html>
    <html>
      <head>
          <title class="main-title">Welcome!</title>
          </head>
      <body>
        <header class="header">
        <h1 class="header-title header" id='welcome'>Welcome to Fruit Page!</h1>
        <div class="main-fruit">
            <ul>
                <li class="fruit-item">Winter_Fruit</li>
                <li class="fruit-item">Spring_Fruit</li>
                <li class="fuit-item">Summer_Fruit</li>
                <li class="fruit-item">Autumn_Fruit</li>
            </ul>
        </nav>
    </header>
    <div class='description main-description'>
      this is a list of fruits
    </div>
    <ul>
        <li>Orange</li>
        <li>Apple</li>
        <li>Tangerine</li>
        <li>Apricot</li>
    </ul>
</body>
</html>"""

soup = BeautifulSoup(example_html, 'html.parser')

## CSS: relazioni gerarchiche

In [3]:
# Select all <li> tags inside a <ul>
li_in_ul = soup.select('ul>li')
print("List items in ul: ")
for element in li_in_ul:
    print(element)

List items in ul: 
<li class="fruit-item">Winter_Fruit</li>
<li class="fruit-item">Spring_Fruit</li>
<li class="fuit-item">Summer_Fruit</li>
<li class="fruit-item">Autumn_Fruit</li>
<li>Orange</li>
<li>Apple</li>
<li>Tangerine</li>
<li>Apricot</li>


## Web dinamico: JavaScript

* le tre componenti principali del web moderno
    * **HTML**: "fornisce il contenuto"
    * **CSS**: "specifica la presentazione del contenuto"
    * **Javascript**: "implementa il comportamento, interagendo con l'utente e con il server"
* JavaScript è usato come linguaggio di programmazione client-side dal **98.5% of di tutti i siti web**
    * fonte <https://w3techs.com/technologies/details/cp-javascript>
    * non possiamo ignorarlo se vogliamo effettuare scraping
* diversi livelli d'uso di **JavaScript**
    * rendere le pagine web più interattive e dinamiche
    * gestioni di semplici form
    * pagine interamente dinamiche in cui tutto il contenuto è generato tramite **JavaScript**

## JavaScript in breve

* rilasciato nel 1995, 6 mesi dopo **JAVA**
* inizialmente utilizzato esclusivamente come linguaggio client-side, eseguito nel browser
* arricchito con **Ajax** nel 1999
* introduzione di **Node.js** nel 2009
    * un run-time environment per **JavaScript** open-source, cross-platform
    * possibilità di esecuzione stand-alone o nel server
* attualmente un linguaggio general purpose

## JavaScript e HTML

* il codice **JavaScript** è incorporato nel documento **HTML** attraverso i tag <code> script </code> <code> /script 
```
<script>
  // Il tuo codice JavaScript è inserito qui!!
</script>
```
oppure si può memorizzare il codice in un file e poi richiamarlo
```
<script src="JSExFiles.js">
</script>
```

* lo script può essere collocato nella **HEAD section** oppure nella **BODY section**

## JavaScript e HTML

* lo script modifica il contenuto della pagina corrente

```
<!DOCTYPE html>
<!-- JSExAlertWrite.html -->
<html lang="en">
<head>
  <meta charset="utf-8">
  <title>JavaScript Example: Functions alert() and document.write()</title>
  <script>
    alert("Hello, world!");
  </script>
</head>
<body>
  <h1>My first JavaScript says:</h1>
  <script>
    document.write("<h2><em>Hello world, again!</em></h2>");
    document.write("<p>This document was last modified on "
        + document.lastModified + ".</p>");
  </script>
</body>
</html>
```

## JavaScript e HTML


<center>
<img src="Figures/JAvaScript.jpg" style="width:500px;height:300px;"/>

* visualizzazione dell'esempio contenuto nella pagina precedente
* il riferimento, contenuto nel codice, a **document** è un riferimento alla **pagina web corrente**, in cui lo script è integrato
* come risultato della operazione <code> document.write() </code>  la **BODY section** del documento ora comprende nuovi tag, registrati in coda al documento
```
<h1>My First JavaScript says</h1>
<h2><em>Hello world, again!</em></h2>
    <p>This document was last modified on mm/dd/yyyy hh:mm:ss.</p>
```
* il codice **JavaScript** ha modificato il layout dell'**HTML**
​

## Quali strumenti per lo scraping di pagine dinamiche?

* **BeautifulSoup** + **requests** non sono più sufficienti
    * utilizzabili solo quando il codice **JavaScript** è semplice ed è possibile un **reverse engineering** del codice
    * capire quale sono le chiamate al server generate da JAvaScript e riprodurle in **BeautifulSoup**
    * molto complesso per la maggior parte delle pagine esistenti
* nel caso più generale sono necessari strumenti più sofisticati:
    * **Selenium**

## Selenium: una libreria per lo scraping di pagine dinamiche

* nasce nel 2004 come tool per lo sviluppo di **test automatici** per applicazioni web
    * possibile scrivere script che simulano l’interazione umana con un'applicazione web
    * da la possibilità di
        * caricare in modo automatico pagine web
        * simulare azioni come quelle compiute da un utente quando interagisce con la pagina
        * con lo scopo di **testare** il comportamento del sito web
* successivamente viene utilizzato come ottimo tool per il **web scraping**
    * scraping per pagine dinamiche, ma con  funzionalità simili a quelle di **BeautifulSoup** per l'individuazione di elementi all'interno della pagina
* binding per **Java**, **Php**, **Python**,...
* non è una libreria Python standard,  installarla  con
    * **pip install -U selenium**

## Selenium: il WebDriver

* per interpretare il codice **JavaScript** occorre un vero interprete del linguaggio, che è disponibile solo all'interno di un browser 
* si utilizza una tecnologia indicata come **WebDriver**
    * standard di **W3C** dal **2018**
    * un'interfaccia diversa di **Webdriver** a seconda del browser utilizzato
    * gli esempi seguenti si riferiranno a **Google Chrome** e **ChromeDriver**
* il WebDriver è avviato da **Selenium** e successivamente **Selenium** può controllare il browser simulando un comportamento umano

## Selenium: il WebDriver

<center>
<img src="Figures/WebDriverBrowser.jpg" style="width:800px;height:400px;"/>
</center>

## Selenium: il WebDriver

* tipi diversi di **WebDriver** a seconda del browser utilizzato
* nel caso di **Chrome**
    * lo **zip** scaricato dalla pagine di **Chrome** contiene un eseguibile di nome **chromedriver.exe**
        * attenzione a scaricare una versione **compatibile** con il vostro browser!
    * **Selenium** deve essere in grado di accedere a questo eseguibile
        * inserire direttamente l'eseguibile nella cartella dove si trova lo script o il Notebook Python
        * oppure indicare esplicitamente  il path 
* **attivazioni** di tipo diverso a seconda del browser

## Selenium: il WebDriver

<code>
from selenium import webdriver
driverChrome = webdriver.Chrome()
driverFirefox = webdriver.FireFox()
driverEdge = webdriver.Edge()
driverSafari = webdriver.Safari()
driverOpera = webdriver.Opera()
driverExplorer = webdriver.Ie()


## Scraping di una pagina con Selenium: https://scrapingclub.com/

<center>
<img src="Figures/ScrapingClub.jpg" style="width:800px;height:500px;"/>
</center>

## Attivazione del webdriver Chrome

In [ ]:
from selenium import webdriver
import time
# lancio del driver di Chrome che fa apparire una pagina vuota del browser
driver=webdriver.Chrome()
# tramite il WebDriver, visita il target site
url='https://scrapingclub.com/'
driver.get(url)
#scraping logic....
time.sleep(100)
#rilascia le risorse allocate da Selenium e shut down del browser
driver.quit()

## Attivazione del webdriver Chrome

* quando si esegue il codice, appare una nuova finestra vuota tipo quella del browser
    * nella finestra viene caricata la pagina richiesta ...con un certo ritardo....
    * **driver.quit()** per chiudere la finestra e rilasciare le risorse
* la pagina è controllata da **Selenium** invece che da un utente, quindi "pilotata" da **Python**
    * infatti si può notare il messaggio **Chrome is being controlled by automated test software**, che avverte che **Selenium** sta controllando questa istanza del browser.
* **Python (Selenium)** può interagire con la pagine con comandi per 
    * inserimento di dati nelle form
    * scrolling della pagina
    * click di buttons
    * e inoltre può analizzare l'HTML della pagina caricata

## Chrome Headless Mode


* **Headless browser** un browser **senza una interfaccia grafica**, ma con tutte le funzionalità di un vero browser
    * utile quando si usa il browser per effettuare **scraping**, oppure per **testare funzionalità** di un sito web
* **Selenium** consente di interagire con un headless browser
    * non vedremo più la finestra di **Chrome**
    * permette di non sprecare risorse nella **GUI**
    * d'altra parte, vedere cosa avviene nel browser  può essere utile in fase di debugging, per osservare gli effetti dello script Python direttamente nel browser
    * in fase di produzione, può essre utile eliminare la finestra
* per attivare la modalità **headless**, occorre settare un oggetto **Options** e passarlo al costruttore del WebDriver

## Chrome Headless Mode

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# enable headless mode in Selenium
options = Options()
options.add_argument('--headless=new')
driver = webdriver.Chrome(
    options=options, 
    # other properties...
)

url='https://scrapingclub.com/'
driver.get(url)
#scraping logic....
print("ho aperto un browser headless")
time.sleep(10)
#rilascia le risorse allocate da Selenium e shut down del browser
driver.quit()


## Selenium: individuare elementi nella pagina HTML

*  **Selenium** offre diversi metodi per reperire gli elementi all'interno di una pagina, simili a quelli di **BeautifulSoup**
* **Attenzione!**
    * in **Selenium 4** il nome di molto metodi è cambiato rispetto a **Selenium 3**
    * molti dei metodi che trovate nei testi di riferimento non sono più validi
    * verificare nome e semantica dei metodi in <https://www.selenium.dev/documentation/overview/> 
    
* due funzioni di base per selezionare elementi **HTML** dal **DOM(Document Object Model)**
    * **find_element:** restituisce un  elemento specifico
    * **find_elements:** restituisce una lista di elementi che soddisfano una condizione
    
* entrambe utilizzano gli attributi della **classe By** come selettori dell'elemento

```
find_element(By.ID, "id")
find_element(By.NAME, "name")
find_element(By.XPATH, "xpath")
find_element(By.LINK_TEXT, "link text")
find_element(By.PARTIAL_LINK_TEXT, "partial link text")
find_element(By.TAG_NAME, "tag name")
find_element(By.CLASS_NAME, "class name")
find_element(By.CSS_SELECTOR, "css selector")
```
* tutti i metodi possono sollevare una **NoSuchElementException**

## Selenium: individuare elementi per Id e per Nome

<center>
<img src="Figures/NameID.jpg" style="width:800px;height:300px;"/>
</center>

## Selenium: individuare elementi per Text e per TagName

<center>
<img src="Figures/TextName.jpg" style="width:800px;height:300px;"/>
</center>

## Selenium: individuare elementi per Class e per CSS Selector

<center>
<img src="Figures/ClassCSS.jpg" style="width:800px;height:300px;"/>
</center>

## Correzione assignment: Scraping Coingecko

<center>
<img src="Figures/coingecko.jpg" style="width:1200px;height:500px;"/>

## Correzione Assignment: Scraping Coingecko

In [3]:
import json
import requests
from bs4 import BeautifulSoup
def fetch_coingecko_html():
    # effettua una richiesta a CoinGecko
    r = requests.get("https://www.coingecko.com/")
    return r 
print(fetch_coingecko_html())

<Response [403]>


* la **risposta del server** è **403**
* l'errore **403 Forbiden (accesso negato)**, è un errore di stato HTTP che indica la mancanza di permessi 
per accedere ai contenuti o a una pagina specifica del sito web
* può essere generato da una strategia **antibot**

## Assignment: Scraping di IMDB, Internet Movie Database

* è un sito web di proprietà di Amazon.com che gestisce informazioni su 
    * film
    * attori
    * registi
    * personale di produzione
    * programmi televisivi
    * videogiochi. 
* considerare la lista di episodi di **Game of Thrones**, che possono essere reperiti in https://www.imdb.com/title/tt0944947/episodes/
* per ogni episodio viene riportata una descrizione, la valutazione media
* gli episodi sono distribuiti su più pagine, per cui serve un processo di **crawling**
* produrre dei grafici che riportino statistiche legate agli episodi, as esempio un bar plor che riporti il rating medio di ogni episodio